In [1]:
# torch imports
import torch
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40
    hard_pred_threshold: 0.5 

log:
    root: '?'
    log_interval: 10 
    log_image_stats: True 
    log_pixel_stats: False 
    gether_inference_stats: True
    summary_compute_global_metrics: False 

dataloader:
    batch_size: '?'
    num_workers: 1 
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

calibrator:
    _name: None 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: '?' 
    num_prob_bins: 15
    neighborhood_width: 3

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg 

# For standard datasets
#####################################
data:
    preload: False 

model:
    pred_label: '?' 
    _type: "standard"
    pretrained_exp_root : None
    ## Use For Uncalibrated
    # checkpoint: 'max-val-dice_score'
    ## Use For Calibrated 
    checkpoint: 'min-val-ece_loss'

<IPython.core.display.Javascript object>

## Gather Inference Options.

In [5]:
%%yaml experiment_cfg 

# Use this field if you want to call it something else beyond the base_model_dir.
name: "ACDC_Experiment_1"


base_model: 
    - "/storage/vbutoi/scratch/ESE/training/07_09_24_ACDC_PixelCELoss/20240709_120223-2YG5-4666f171cc2ff0127caa727844be2be1"
    - "/storage/vbutoi/scratch/ESE/training/07_09_24_ACDC_SoftDiceLoss/20240709_115941-2WD3-0a311a8965d658cbf9bcf19bfaedda25"
    - "/storage/vbutoi/scratch/ESE/calibration/07_12_24_ACDC_CalibratorSet_Experiment1"

data:
    _class: "ese.experiment.datasets.ACDC"
    split: "val"
    label_threshold: null # Need to compare with non-noisy gt.

dataloader:
    batch_size: 8

<IPython.core.display.Javascript object>

In [6]:
# Local imports
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])
# Get the different experiment cfg yamls.
inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

In [7]:
len(inf_cfgs)

18

## Running Jobs

In [8]:
from ese.experiment.analysis.run_inference import get_cal_stats

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
# from ese.experiment.experiment import run_ese_exp

# ###### Run individual jobs
# run_ese_exp(
#     config=inf_cfgs[0], 
#     job_func=get_cal_stats,
#     run_name='debug',
#     # run_name=exp_name,
#     show_examples=True,
#     gpu='0',
#     # gpu='4',
# ) 

In [10]:
from ese.experiment.experiment import submit_ese_exps 

#### Run Batch Jobs
submit_ese_exps(
    config_list=inf_cfgs,
    job_func=get_cal_stats,
    available_gpus=['0', '1', '2', '3']
    # available_gpus=['4', '5', '6', '7']
)

Submitted job id: 1434966 on gpu: 0.
Submitted job id: 1435069 on gpu: 1.
Submitted job id: 1435244 on gpu: 2.
Submitted job id: 1435413 on gpu: 3.
Submitted job id: 1435582 on gpu: 0.
Submitted job id: 1435750 on gpu: 1.
Submitted job id: 1435919 on gpu: 2.
Submitted job id: 1436087 on gpu: 3.
Submitted job id: 1436256 on gpu: 0.
Submitted job id: 1436429 on gpu: 1.
Submitted job id: 1436599 on gpu: 2.
Submitted job id: 1436768 on gpu: 3.
Submitted job id: 1436956 on gpu: 0.
Submitted job id: 1437125 on gpu: 1.
Submitted job id: 1437293 on gpu: 2.
Submitted job id: 1437461 on gpu: 3.
Submitted job id: 1437629 on gpu: 0.
Submitted job id: 1437797 on gpu: 1.
